# Training a sentiment analysis classifier based on supervised machine learning algorithms

In [62]:
import string

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 


from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score ,confusion_matrix

In [63]:
print("entro")

entro


In [64]:
pd.set_option('display.max_colwidth', None)

In [65]:
stop_words = set(stopwords.words('spanish'))

In [66]:
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

### Loading labeled tweets

In [67]:
# Dataset loaded from: https://docs.google.com/spreadsheets/d/11_E2ngsEOyTQwbwVVRHY5urzFF95BQCV/edit#gid=1788161364
tweets_df = pd.read_csv('./data/tweets_labeled.csv', sep = ',')

In [68]:
tweets_df.shape

(648, 3)

In [69]:
tweets_df.head()

,full_text,sentiment,emotion
0,"@IvanDuque Sus socios de las AGC tienen este platanal vuelto mierda, pero no haces nada, usted inservible, ha logrado la peor inflación en Colombia y solo pública maricadas sin sentido, se robó las elecciones para entregar el país, valiente idiota tenemos de presidente.",negative,anger
1,@AlvaroUribeVel Ellos también celebran por que Maduro PetroGustavo Ivan Cepeda Timo Teodora Bobolivar entre muchos más sufren de insomnio.\r\nUribe Velez Alvarito no los deja dormir\r\nEl enemigo para las elecciones no es Fico.... Sino el Excelentísimo y más Grande Presidente que ha tenido Colombia,negative,anger
2,"@PDleh @cokycafe @PGN_COL En Colombia existen miles de leyes para controlar los funcionarios públicos pero a la hora de aplicarlas ""se les olvida que existen"". ¿Que más pruebas quieren? ¿Porque avalan unas elecciones, las del 13 de marzo, donde pado de todo lo que no debía de pasar? Escuchamos respuestas",negative,anger
3,Petro ahora es el Capitán América o mejor dicho el Capitán Colombia de ganar las elecciones. No soltara el escudo ni para dormir 😂 https://t.co/k56Dv7id1J,negative,joy
4,"#OtraPregunta \r\n""Petro promete reanudar relaciones diplomáticas con Maduro si gana elecciones en Colombia""...\r\nEsto no pinta nada bien... y si además gana Lula en Brasil...\r\n¿Esto como que va para largo? https://t.co/wpQsl5KoRe",negative,fear


In [70]:
tweets_df['sentiment'].value_counts(dropna = False, normalize = True)

negative    0.404321
NaN         0.395062
neutral     0.166667
positive    0.033951
Name: sentiment, dtype: float64

In [71]:
tweets_df['emotion'].value_counts(dropna = False, normalize = True)

NaN        0.444444
anger      0.268519
joy        0.135802
fear       0.104938
sadness    0.046296
Name: emotion, dtype: float64

### Leaving out unlabeled texts, this data is not useful for training or validating a supervised model

In [72]:
# Removing  unlabeled tweets
tweets_labeled_df = tweets_df.loc[tweets_df['sentiment'].notnull()]

In [73]:
tweets_labeled_df.shape

(392, 3)

In [74]:
tweets_unlabeled_df = tweets_df.loc[tweets_df['sentiment'].isnull()]

In [75]:
tweets_unlabeled_df.shape

(256, 3)

In [76]:
# Scenario 1: Working with all classes

# Scenario 2: Working only with positive and negative classes
# Removing neutral class
#tweets_labeled_df = tweets_labeled_df.loc[tweets_labeled_df['sentiment'].isin(['positive', 'negative'])]

# Scenario 3: Treating neutral as positive classes
tweets_labeled_df['sentiment'] = tweets_labeled_df['sentiment'].replace({'neutral': 'positive'})

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_10272/778818246.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_labeled_df['sentiment'] = tweets_labeled_df['sentiment'].replace({'neutral': 'positive'})


### Splitting train and test datasets

In [77]:
X_train, X_test, y_train, y_test = train_test_split(tweets_labeled_df['full_text'], tweets_labeled_df['sentiment'], test_size = 0.2, stratify = tweets_labeled_df['sentiment'], random_state = 1)


In [78]:
X_train.shape

(313,)

In [79]:
pd.Series(y_train).value_counts(normalize = True)

negative    0.667732
positive    0.332268
Name: sentiment, dtype: float64

In [80]:
X_test.shape

(79,)

In [81]:
pd.Series(y_test).value_counts(normalize = True)

negative    0.670886
positive    0.329114
Name: sentiment, dtype: float64

### Vectorizing texts

In [82]:
bow = CountVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [83]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [84]:
X_bow = bow.fit_transform(X_train)

In [85]:
X_tfidf = tfidf.fit_transform(X_train)

### Training and evaluating a model using BOW

In [86]:
model = DecisionTreeClassifier()

In [87]:
model.fit(X_bow, y_train)

DecisionTreeClassifier()

In [88]:
y_train_bow_predict = model.predict(X_bow)
y_test_bow_predict = model.predict(bow.transform(X_test))

In [89]:


predictions = train_test_split.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,   display_labels=clf.classes_)
disp.plot()

AttributeError: 'function' object has no attribute 'predict'

In [90]:
# Metrics calculation for binary classes
print('Precision:', precision_score(y_test, y_test_bow_predict, pos_label = 'positive'))
print('Recall:', recall_score(y_test, y_test_bow_predict, pos_label = 'positive'))
print('F1:', f1_score(y_test, y_test_bow_predict, pos_label = 'positive'))

Precision: 0.4782608695652174
Recall: 0.4230769230769231
F1: 0.44897959183673475


In [91]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_bow_predict, average = None))
print('Recall:', recall_score(y_test, y_test_bow_predict, average = None))
print('F1:', f1_score(y_test, y_test_bow_predict, average = None))

Precision: [0.73214286 0.47826087]
Recall: [0.77358491 0.42307692]
F1: [0.75229358 0.44897959]


### Training and evaluating a model using TF-IDF

In [92]:
model = RandomForestClassifier()

In [93]:
model.fit(X_tfidf, y_train)

RandomForestClassifier()

In [94]:
y_train_tfidf_predict = model.predict(X_tfidf)
y_test_tfidf_predict = model.predict(bow.transform(X_test))

In [95]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_tfidf_predict)


AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [96]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_tfidf_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [97]:
# Metrics calculation for binary classes
print('Precision:', precision_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))
print('Recall:', recall_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))
print('F1:', f1_score(y_test, y_test_tfidf_predict, pos_label = 'positive'))

Precision: 0.5333333333333333
Recall: 0.6153846153846154
F1: 0.5714285714285715


In [98]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_tfidf_predict, average = None))
print('Recall:', recall_score(y_test, y_test_tfidf_predict, average = None))
print('F1:', f1_score(y_test, y_test_tfidf_predict, average = None))

Precision: [0.79591837 0.53333333]
Recall: [0.73584906 0.61538462]
F1: [0.76470588 0.57142857]


### How interpret the results?

### Analyzing errors

In [99]:
error_df = pd.concat(
    [ pd.concat([X_test, y_test ], axis = 1).reset_index(),
    pd.Series(y_test_bow_predict) ]
, axis = 1).rename(columns = { 'sentiment': 'actual', 0: 'predicted' })

error_df.drop('index', inplace = True, axis = 1)

In [100]:
error_df.shape

(79, 3)

In [101]:
error_df.loc[error_df['actual'] != error_df['predicted']].head()

,full_text,actual,predicted
0,"@GustavoBolivar Debieran inventar algo diferente están igual que Chávez y Maduro cada vez que vienen elecciones inventaba guerras con Colombia, que había intereses en matarlo y ustedes igualito inventan enfermedades, atentados etc, etc.",positive,negative
1,En Colombia hace falta un presidente como @nayibbukele que no le teme a nada y va de frente contra las fuerzas oscuras aliadas con políticos que lo único que buscas es inestabilidad en el país previo a elecciones.,positive,negative
3,En Colombia con las elecciones,positive,negative
6,"La cuenta regresiva para las elecciones presidenciales de Colombia se encuentra en su recta final, y el candidato que cuenta de acuerdo a la encuestas con mayor intención de votos es Gustavo Petro #7May https://t.co/JVyQw3MCv7",positive,negative
7,"Con la extradición de Otoniel, se llevaron un posible muy apoyado candidato a la Presidencia de Colombia en las próximas elecciones.",negative,positive


In [102]:
print("entr")

entr
